In [1]:
from rdkit import Chem
#from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
from rdkit.Chem.Draw import IPythonConsole
import rdkit
import time
import pandas as pd
print(rdkit.__version__)
print(time.asctime())

2022.09.1
Wed May  3 14:51:50 2023


In [2]:
import ipywidgets as widgets
from traitlets import Unicode, Int, validate
class MolSVGWidget(widgets.DOMWidget):
    _view_name = Unicode('MolSVGView').tag(sync=True)
    _view_module = Unicode('molsvg_widget').tag(sync=True)
    _view_module_version = Unicode('0.0.1').tag(sync=True)
     
    svg = Unicode('', help="svg to be rendered").tag(sync=True)
    #selected_atoms = Unicode('', help="list of currently selected atoms").tag(sync=True)
    clicked_atom_idx = Unicode('', help="The index of the atom that was just clicked").tag(sync=True)


In [1]:
%%js
alert("hello, world!");

<IPython.core.display.Javascript object>

In [8]:
%%javascript
// make sure our module is only defined
// only once.
require.undef('molsvg_widget');
 
// Define the `molsvg_widget` module using the Jupyter widgets framework.
define('molsvg_widget', ["@jupyter-widgets/base"],
       function(widgets) {
 
    // The frontend class:
    var MolSVGView = widgets.DOMWidgetView.extend({
 
        // This method creates the HTML widget.
        render: function() {
            this.svg_div = document.createElement('div');
            this.el.appendChild(this.svg_div);
            this.model.on('change:svg', this.svg_changed, this);
            this.svg_changed();
        },
         
        // called when the SVG is updated on the Python side
        svg_changed: function() {
            var txt = this.model.get('svg'); 
            this.svg_div.innerHTML = txt;
            var sels = this.svg_div.getElementsByClassName("atom-selector");
            for(var i=0;i&amp;amp;lt;sels.length;i++){ sels[i].onclick = (evt) =&amp;amp;gt; { return this.atom_clicked(evt) };
                //sels[i].r = sels[i].r*2; #R is read only, set_r?
                //Or regexp the r from the svg and increase the size there.
            }
             
        },
 
        // callback for when an atom is clicked
        atom_clicked: function(evt) {
            //alert("  "+evt+"|"+this);
            if(!evt.currentTarget.getAttribute('class')){
                return;
            }
            var satmid = evt.currentTarget.getAttribute('class').match(/atom-([0-9]+)/);
            if(satmid.length &amp;amp;gt;1){
                var atmid = Number(satmid[1]);
                //var curSel = this.model.get('selected_atoms');
                //var splitSel = curSel.split(',');
                //var selItms = [];
                //var idx = -1;
                //alert("|"+atmid+"|"+curSel+"|len: "+splitSel.length);
                //if(curSel != "" &amp;amp;amp;&amp;amp;amp; splitSel.length&amp;amp;gt;0){
                //    selItms = Array.from(splitSel).map(item =&amp;amp;gt; Number(item));
                //    idx = selItms.indexOf(atmid);
                //}
                //if(idx == -1){
                //    selItms = selItms.concat(atmid);
                //    evt.currentTarget.style["stroke-width"]=3;
                //    evt.currentTarget.style["stroke-opacity"]=1;
                //    evt.currentTarget.style["stroke"]='#AA22FF';
                //} else {
                //    selItms.splice(idx,1);
                //    evt.currentTarget.style["stroke-width"]=1;
                //    evt.currentTarget.style["stroke-opacity"]=0;
                //   evt.currentTarget.style["stroke"]='#FFFFFF';
                //}
                //this.model.set('selected_atoms',String(selItms));
                this.model.set('clicked_atom_idx',"event_hack");
                this.touch();
                this.model.set('clicked_atom_idx',String(atmid));
                this.touch();
            }
        }
 
    });
 
    return {
        MolSVGView : MolSVGView
    };
});

<IPython.core.display.Javascript object>

In [8]:
from IPython.display import display, Javascript

# Define the JavaScript code to create a button widget
js_button = """
var button = document.createElement('button');
button.innerHTML = 'Click me!';
button.onclick = function() {
    alert('Button clicked!');
};
element.append(button);
"""

# Create a display element to hold the widget
button_display = display('asdasd', display_id=True)

# Display the JavaScript code in the display element
button_display.update(Javascript(js_button))


In [5]:
o = widgets.Output()
display(o)
with o:
    print("Hello RDKittens!")
with o:
    print("Hello RDKids!")

Output()

In [12]:
class AnnotateMol(object):
    def __init__(self, mol = Chem.MolFromSmiles("c1c([NH3+])cccc1CC(=O)O")):
         
        style = {'description_width': 'initial'}
        #Create the outputs and widgets
        self.o_mol = widgets.Output()
        self.o_molstring = widgets.Output()
        self.o_table = widgets.Output()
        self.o_atomclicked = widgets.Text(description="Index of clicked atom",
                                         #layout = widgets.Layout(width="100px"),
                                         style=style)
        self.t_propertyname = widgets.Text(description="Property Name", style=style)
        self.t_propertyvalue =widgets.Text(description="Property Value", style=style)
        #Make the GUI
         
        display(widgets.HBox([self.t_propertyname, self.t_propertyvalue]))
        display(self.o_atomclicked)
        display(widgets.HBox([self.o_mol, self.o_table]))
         
        #Set the mol
        self.mol = mol
 
    @property
    def mol(self):
        """Return the private mol"""
        return self._mol
     
    @mol.setter
    def mol(self, mol):
        """Set the private mol and initalize interactive SVG and update output widgets"""
        self._mol = mol
        self.create_widget()
        self.draw_widget()
         
         
    def create_widget(self):
        """Create the interactive SVG mol widget"""
        d = rdMolDraw2D.MolDraw2DSVG(200,150)
        dm = Draw.PrepareMolForDrawing(self.mol)
        d.DrawMolecule(dm)
        d.TagAtoms(dm)
        d.FinishDrawing()
        svg = d.GetDrawingText()
        # self.w = MolSVGWidget(svg=svg)
        self.w = "Hello RDKittens!"
         
    def draw_widget(self):
        """Display the mol widget"""
        self.o_mol.clear_output()
        with self.o_mol:
            display(self.w)
 
app = AnnotateMol()

Text(value='', description='Index of clicked atom', style=TextStyle(description_width='initial'))

In [7]:
mol2 = Chem.MolFromSmiles("CCCCN(C)CCCC")
mol2.GetAtomWithIdx(4).SetProp("pKa","12.4")
mol2.GetAtomWithIdx(5).SetProp("molFileValue","Hello SD-file!")
app.mol = mol2